In [220]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline 
import sys
from sklearn.preprocessing import normalize

In [201]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\new_data.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
y_data=xy[:,[-1]]
m_data=xy[:499,:]

F_data=xy[:,:]

In [223]:
def plot(samples, D_loss, G_loss, epoch, total):

    fig = plt.figure(figsize=(10, 5))        

    gs = gridspec.GridSpec(4, 8)
    gs.update(wspace=0.05, hspace=0.05)
    
    # Plot losses
    ax = plt.subplot(gs[:, 4:])
    ax.plot(D_loss, label="discriminator's loss", color='b')
    ax.plot(G_loss, label="generator's loss", color='r')
    ax.set_xlim([0, total])
    ax.yaxis.tick_right()
    ax.legend()
    for i, sample in enumerate(samples):

        if i > 4* 4 - 1:
            break
        ax = plt.subplot(gs[i % 4, int(i / 4)])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.savefig('./output/' + str(epoch + 1) + '.png')
    plt.close()

In [226]:
  # Generate images
    

def Conv2d(input, output_dim=64, kernel=(5, 5), strides=(2, 2), stddev=0.2, name='conv_2d'):

    with tf.variable_scope(name):
        W = tf.get_variable('Conv2dW', [kernel[0], kernel[1], input.get_shape()[-1], output_dim],
                             initializer=tf.truncated_normal_initializer(stddev=stddev))
        b = tf.get_variable('Conv2db', [output_dim], initializer=tf.zeros_initializer())
        
        return tf.nn.conv2d(input, W, strides=[1, strides[0], strides[1], 1], padding='SAME') + b

def Deconv2d(input, output_dim, batch_size, kernel=(5, 5), strides=(2, 2), stddev=0.2, name='deconv_2d'):
    
    with tf.variable_scope(name):
        W = tf.get_variable('Deconv2dW', [kernel[0], kernel[1], output_dim, input.get_shape()[-1]],
                             initializer=tf.truncated_normal_initializer(stddev=stddev))
        b = tf.get_variable('Deconv2db', [output_dim], initializer=tf.zeros_initializer())

        input_shape = input.get_shape().as_list()
        output_shape = [batch_size,
                        int(input_shape[1] * strides[0]),
                        int(input_shape[2] * strides[1]),
                        output_dim]

        deconv = tf.nn.conv2d_transpose(input, W, output_shape=output_shape,
                                        strides=[1, strides[0], strides[1], 1])
    
        return deconv + b

def Dense(input, output_dim, stddev=0.02, name='dense'):
    
    with tf.variable_scope(name):
        
        shape = input.get_shape()
        W = tf.get_variable('DenseW', [shape[1], output_dim], tf.float32,
                            tf.random_normal_initializer(stddev=stddev))
        b = tf.get_variable('Denseb', [output_dim],
                            initializer=tf.zeros_initializer())
        
        return tf.matmul(input, W) + b

def BatchNormalization(input, name='bn'):
    
    with tf.variable_scope(name):
    
        output_dim = input.get_shape()[-1]
        beta = tf.get_variable('BnBeta', [output_dim],
                            initializer=tf.zeros_initializer())
        gamma = tf.get_variable('BnGamma', [output_dim],
                            initializer=tf.ones_initializer())
    
        if len(input.get_shape()) == 2:
            mean, var = tf.nn.moments(input, [0])
        else:
            mean, var = tf.nn.moments(input, [0, 1, 2])
        return tf.nn.batch_normalization(input, mean, var, beta, gamma, 1e-5)
    
def LeakyReLU(input, leak=0.2, name='lrelu'):
    
    return tf.maximum(input, leak*input)



In [231]:
def Discriminator(X, reuse=False, name='d'):

    with tf.variable_scope(name, reuse=reuse):

        if len(X.get_shape()) > 2:
            # X: -1, 28, 28, 1
            D_conv1 = Conv2d(X, output_dim=64, name='D_conv1')
        else:
            D_reshaped = tf.reshape(X, [-1, 1, 42, 1])
            D_conv1 = Conv2d(D_reshaped, output_dim=64, name='D_conv1')
        D_h1 = LeakyReLU(D_conv1) # [-1, 28, 28, 64]
        D_conv2 = Conv2d(D_h1, output_dim=128, name='D_conv2')
        D_h2 = LeakyReLU(D_conv2) # [-1, 28, 28, 128]
        D_r2 = tf.reshape(D_h2, [-1, 256])
        D_h3 = LeakyReLU(D_r2) # [-1, 256]
        D_h4 = tf.nn.dropout(D_h3, 0.5)
        D_h5 = Dense(D_h4, output_dim=1, name='D_h5') # [-1, 1]
        return tf.nn.sigmoid(D_h5)

def Generator(z, name='g'):

    with tf.variable_scope(name):

        G_1 = Dense(z, output_dim=1024, name='G_1') # [-1, 1024]
        G_bn1 = BatchNormalization(G_1, name='G_bn1')
        G_h1 = tf.nn.relu(G_bn1)
        G_2 = Dense(G_h1, output_dim=7*7*128, name='G_2') # [-1, 7*7*128]
        G_bn2 = BatchNormalization(G_2, name='G_bn2')        
        G_h2 = tf.nn.relu(G_bn2)
        G_r2 = tf.reshape(G_h2, [-1, 7, 7, 128])
        G_conv3 = Deconv2d(G_r2, output_dim=64, batch_size=1, name='G_conv3')
        G_bn3 = BatchNormalization(G_conv3, name='G_bn3')        
        G_h3 = tf.nn.relu(G_bn3)
        G_conv4 = Deconv2d(G_h3, output_dim=1, batch_size=1, name='G_conv4')
        G_r4 = tf.reshape(G_conv4, [-1, 42])
        return tf.nn.sigmoid(G_r4)


In [233]:
X = tf.placeholder(tf.float32, shape=[None, 42])
z = tf.placeholder(tf.float32, shape=[None, 21])

G = Generator(z, 'G')
D_real = Discriminator(X, False, 'D')
D_fake = Discriminator(G, True, 'D')

D_loss = -tf.reduce_mean(tf.log(D_real) - tf.log(D_fake)) # Train to judge if the data is real correctly
G_loss = -tf.reduce_mean(tf.log(D_fake)) # Train to pass the discriminator as real data

vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('D/')]
g_params = [v for v in vars if v.name.startswith('G/')]

D_solver = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.1).minimize(D_loss, var_list=d_params)
G_solver = tf.train.AdamOptimizer(learning_rate=2e-4, beta1=0.3).minimize(G_loss, var_list=g_params)

ValueError: Variable G/G_1/DenseW already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "C:\Users\SANHA\Anaconda3\envs\py35\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [218]:
samples
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('gen_sample.txt', 'w')
    for step in range(500):
        
        
        batch_images = F_data[step].reshape([1, 42])
        batch_z = np.random.uniform(0, 1, size=[1, z_dim])
        _, D_loss_curr = sess.run([D_solver, D_loss], {X: batch_images, z: batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        
        sample_z = np.random.uniform(0, 1, size=[1, z_dim])  # 16 Samples each epoch
        _, G_loss_curr = sess.run([G_solver, G_loss], {z: sample_z})
        
        D_loss_vals.append(D_loss_curr)
        G_loss_vals.append(G_loss_curr)
        if step%10==0:
            print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 10000, loss_d, loss_g))
        
        
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        gen_dd = gen_samples.resize([1, 42])
        gen_re=np.uniform(gen_samples,(1,42))
        f.write(str(gen_dd))
        samples.append(gen_dd)

    print('Training Complete. ')
    f.close()

step 1 / 10000 Complete. D_Loss : 8.680, G_Loss : 2.124


AttributeError: 'numpy.ndarray' object has no attribute 'uniform'

In [219]:
m_data[12].ndim
samples[499].ndim
m_data=np.vstack((m_data, samples))

ValueError: all the input arrays must have same number of dimensions

In [178]:
#xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
#x_data=xy[:,0:-1]
#y_data=xy[:,[-1]]
#F_data=xy[:,:]


x1_data=m_data[:,0:-1]
y1_data=m_data[:,[-1]]



nb_classes=2

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,128]),name='weight1')
b1=tf.Variable(tf.random_normal([128]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([128,128]),name='weight2')
b2=tf.Variable(tf.random_normal([128]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([128,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        sess.run(optimizer,feed_dict={X:x1_data,Y:y1_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:x_data,Y:y_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
  
    tr=0
    fa=0
    total=0


    pred = sess.run(prediction, feed_dict={X: x_data})
    for p, y in zip(pred, y_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
            #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))

    print("true={} false: {} acc: {:0.2f}%".format(tr,fa,tr/(tr+fa)))

step:     0	Loss: 4.760	Acc: 53.46%
step:    10	Loss: 3.048	Acc: 52.58%
step:    20	Loss: 2.528	Acc: 50.55%
step:    30	Loss: 2.470	Acc: 50.45%
step:    40	Loss: 2.466	Acc: 50.55%
step:    50	Loss: 2.456	Acc: 50.59%
step:    60	Loss: 2.432	Acc: 50.58%
step:    70	Loss: 2.401	Acc: 50.50%
step:    80	Loss: 2.382	Acc: 50.51%
step:    90	Loss: 2.361	Acc: 50.49%
step:   100	Loss: 2.339	Acc: 50.45%
step:   110	Loss: 2.318	Acc: 50.41%
step:   120	Loss: 2.297	Acc: 50.35%
step:   130	Loss: 2.276	Acc: 50.30%
step:   140	Loss: 2.267	Acc: 50.31%
step:   150	Loss: 2.250	Acc: 50.25%
step:   160	Loss: 2.231	Acc: 50.20%
step:   170	Loss: 2.204	Acc: 50.10%
step:   180	Loss: 2.181	Acc: 50.03%


KeyboardInterrupt: 